# 모든 model에 대해 각각 model training + 5-fold + SHAP -> 비교
- catBoost, XGBoost, Randomforest, Deepforest 총 4가지 모델 사용

- feature들 간의 상관계수를 계산하는 방법은 오직 선형성만 파악하고 각 인자의 중요도는 파악하기 어렵기 때문에 SHAP를 사용함
- 모델마다 각 인자의 중요도를 해석하는 기준과 방식이 다르고 동일한 인자라도 모델에 따라 영향력을 다르게 판단하기 때문에 각 모델마다 중요도 분석을 진행함
ex) Catboost : 범주형 인자를 잘 처리하므로 이 인자의 중요도가 높게 나올 수 有
    XGBoost : tree split 기준에서 빠르게 좋은 gain을 얻는 인자에 편향될 수 有
    Randomforest : 다양한 tree에 균등하게 기여한 인자에 중요도를 부여함


In [ ]:
import numpy as np
import shap

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from deepforest import CascadeForestRegressor
from sklearn.model_selection import KFold

# 1) 모델 팩토리 정의
model_factories = {
    "CatBoost":     lambda: CatBoostRegressor(verbose=0, random_state=42),
    "XGBoost":      lambda: XGBRegressor(use_label_encoder=False, eval_metric="rmse", random_state=42),
    "RandomForest": lambda: RandomForestRegressor(n_estimators=100, random_state=42),
    "DeepForest":   lambda: CascadeForestRegressor(random_state=42)
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)
feature_names = X.columns.tolist()

# 2) k‑fold + 함수형 SHAP 실행
for name, factory in model_factories.items():
    print(f"🚀 Processing {name}...")
    shap_values_all = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # ✨ 매 fold마다 새 모델 생성
        model = factory()
        model.fit(X_train, y_train)

        try:
            explainer = shap.Explainer(
                model.predict,
                X_train,
                feature_names=feature_names,
                algorithm="partition"
            )
            explanation = explainer(X_test)
            shap_vals = explanation.values    # (n_samples, n_features)
            shap_values_all.append(np.abs(shap_vals))
        except Exception as e:
            print(f"❌ SHAP failed for {name} (Fold {fold}): {e}")

    if shap_values_all:
        all_shap = np.vstack(shap_values_all)
        mean_shap = all_shap.mean(axis=0)
        print(
            f"✅ {name} mean | " +
            ", ".join(f"{f}:{m:.4f}" for f, m in zip(feature_names, mean_shap))
        )
    else:
        print(f"⚠️ {name} 에 대해 SHAP 값이 하나도 계산되지 않았습니다.")

🚀 Processing CatBoost...


Partition explainer: 207it [00:17,  5.12it/s]                         
Partition explainer: 207it [00:11,  2.30it/s]                         
Partition explainer: 207it [00:10,  1.39it/s]                         
[16:11:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



✅ CatBoost mean | cement:7.1196, blast furnace slag:3.6854, fly ash:0.4022, water :4.3894, superplasticizer:2.0421, coarse aggregate:0.9427, fine aggregate:1.4503, age:7.3261
🚀 Processing XGBoost...


[16:11:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

Partition explainer: 207it [00:13,  4.31it/s]                         
[16:11:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

Partition explainer: 207it [00:26,  4.87it/s]                         
[16:12:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

Partition explainer: 207it [00:16,  4.84it/s]                         
[16:12:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters

✅ XGBoost mean | cement:7.1339, blast furnace slag:3.6595, fly ash:0.4803, water :3.8367, superplasticizer:2.4300, coarse aggregate:0.8130, fine aggregate:1.4693, age:7.8960
🚀 Processing RandomForest...


...

[2025-07-28 07:18:05.790] Start to evalute the model:
[2025-07-28 07:18:05.791] Evaluating cascade layer = 0 
[2025-07-28 07:18:05.812] Evaluating cascade layer = 1 
[2025-07-28 07:18:05.832] Start to evalute the model:
[2025-07-28 07:18:05.832] Evaluating cascade layer = 0 
[2025-07-28 07:18:05.847] Evaluating cascade layer = 1 
[2025-07-28 07:18:05.904] Start to evalute the model:
[2025-07-28 07:18:05.905] Evaluating cascade layer = 0 
[2025-07-28 07:18:05.934] Evaluating cascade layer = 1 
[2025-07-28 07:18:05.958] Start to evalute the model:
[2025-07-28 07:18:05.958] Evaluating cascade layer = 0 
[2025-07-28 07:18:05.998] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.056] Start to evalute the model:
[2025-07-28 07:18:06.056] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.110] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.162] Start to evalute the model:
[2025-07-28 07:18:06.162] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.215] Evaluating cascade layer = 1 
[202

Partition explainer:  75%|███████▌  | 155/206 [01:35<00:33,  1.54it/s]

[2025-07-28 07:18:06.459] Start to evalute the model:
[2025-07-28 07:18:06.460] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.475] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.493] Start to evalute the model:
[2025-07-28 07:18:06.493] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.498] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.519] Start to evalute the model:
[2025-07-28 07:18:06.519] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.536] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.556] Start to evalute the model:
[2025-07-28 07:18:06.557] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.595] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.643] Start to evalute the model:
[2025-07-28 07:18:06.643] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.692] Evaluating cascade layer = 1 
[2025-07-28 07:18:06.746] Start to evalute the model:
[2025-07-28 07:18:06.746] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.795] Evaluating cascade layer = 1 
[202

Partition explainer:  76%|███████▌  | 156/206 [01:35<00:31,  1.60it/s]

[2025-07-28 07:18:06.950] Start to evalute the model:
[2025-07-28 07:18:06.951] Evaluating cascade layer = 0 
[2025-07-28 07:18:06.984] Evaluating cascade layer = 1 
[2025-07-28 07:18:07.029] Start to evalute the model:
[2025-07-28 07:18:07.029] Evaluating cascade layer = 0 
[2025-07-28 07:18:07.047] Evaluating cascade layer = 1 
[2025-07-28 07:18:07.063] Start to evalute the model:
[2025-07-28 07:18:07.063] Evaluating cascade layer = 0 
[2025-07-28 07:18:07.071] Evaluating cascade layer = 1 
[2025-07-28 07:18:07.078] Start to evalute the model:
[2025-07-28 07:18:07.078] Evaluating cascade layer = 0 
[2025-07-28 07:18:07.094] Evaluating cascade layer = 1 
[2025-07-28 07:18:07.158] Start to evalute the model:


In [12]:
import pandas as pd
import copy
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from deepforest import CascadeForestRegressor
import numpy as np
np.int = int
np.bool = bool

In [13]:
df = pd.read_csv("dataset.csv")
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

In [14]:
# 사용할 모델 정의
models = {
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
    "XGBoost":  XGBRegressor(use_label_encoder=False, eval_metric="rmse", random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "DeepForest": CascadeForestRegressor(random_state=42)
}

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
feature_names = X.columns.tolist()

In [21]:

# 2) 각 모델별로 k‑fold 돌면서 SHAP 계산
for name, factory in model_factories.items():
    print(f"🚀 Processing {name}...")
    shap_values_all = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # 매 fold마다 새 모델 인스턴스 생성
        model = factory()
        model.fit(X_train, y_train)

        try:
            # 함수형 PartitionExplainer 우회 방식
            explainer = shap.Explainer(
                model.predict,
                X_train,
                feature_names=feature_names,
                algorithm="partition"
            )
            explanation = explainer(X_test)
            shap_vals = explanation.values    # shape = (n_samples, n_features)
            shap_values_all.append(np.abs(shap_vals))
        except Exception as e:
            print(f"❌ SHAP failed for {name} (Fold {fold}): {e}")

    # 결과 집계
    if shap_values_all:
        all_shap = np.vstack(shap_values_all)
        mean_shap = all_shap.mean(axis=0)
        print(
            f"✅ {name} mean | " +
            ", ".join(f"{f}:{m:.4f}" for f, m in zip(feature_names, mean_shap))
        )
    else:
        print(f"⚠️ {name} 에 대해 SHAP 값이 하나도 계산되지 않았습니다.")

NameError: name 'model_factories' is not defined

In [18]:
# 결과 table로 나타내기
importance_df = pd.DataFrame(results)
importance_df = importance_df.loc[importance_df.mean(axis=1).sort_values(ascending=False).index]

print("\nFeature Importance Comparison (Mean SHAP Values across 5 folds):")
print(importance_df)



Feature Importance Comparison (Mean SHAP Values across 5 folds):
                   CatBoost XGBoost  RandomForest DeepForest
age                    None    None      7.870133       None
cement                 None    None      7.172226       None
blast furnace slag     None    None      3.307612       None
water                  None    None      3.059766       None
superplasticizer       None    None      2.570701       None
fine aggregate         None    None      1.126288       None
coarse aggregate       None    None      0.437356       None
fly ash                None    None      0.277504       None


In [ ]:
# 시각화를 위해 마지막 fold 기준 summary plot 그려보기
for model_name, model in models.items():
    print(f"\n📈 SHAP Summary Plot for {model_name}")
    
    # DeepForest는 SHAP 미지원 → 건너뜀
    if model_name == "DeepForest":
        print("⚠️ SHAP does not support DeepForest. Skipping.")
        continue

    # 모델 학습
    model.fit(X, y)

    # SHAP 분석 (TreeExplainer 사용)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    # 시각화
    shap.summary_plot(shap_values, X, feature_names=X.columns)


📈 SHAP Summary Plot for CatBoost
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\SAL\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SAL\AppData\Local\Temp\ipykernel_5812\2589036907.py", line 18, in <module>
    shap.summary_plot(shap_values, X, feature_names=X.columns)
  File "c:\Users\SAL\AppData\Local\Programs\Python\Python38\lib\site-packages\shap\__init__.py", line 31, in unsupported
ImportError: matplotlib is not installed so plotting is not available! Run `pip install matplotlib` to fix this.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SAL\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\SAL\AppData\Roaming\Python\Python38\site-packages\IPython\core\ultratb.py", line 1396, in stru

모든 fold의 SHAP summary plot을 그리는 것은 많은 연산이 필요하고 코드도 꽤나 복잡하기 때문에 그 중 하나만 대표적으로 보여주는 의미로 마지막  fold 기준을 사용함
-> test set이 전체 데이터를 잘 반영한다면 하나의 fold를 보더라도 중요한 인자의 경향은 비슷하게 나오고 전체적인 중요 feature 순위나 분포를 대부분 유사하므로 하나만 사용해도 괜찮을 것 같다고 판단